In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.vgg19 import VGG19
from keras.applications.imagenet_utils import preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexEntropy"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = VGG19(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 512)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 512)

In [ ]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [ ]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   57.4s


[CV] ........................... C=0.01, score=0.911968, total=  57.2s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.912402, total=  57.6s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.921315, total=  57.6s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.910156, total=  59.9s
[CV] C=1.0 ...........................................................
[CV] ............................ C=0.1, score=0.907227, total= 1.1min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.918731, total= 1.1min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.927165, total= 1.1min
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  1.2min


[CV] ............................ C=0.1, score=0.916830, total= 1.2min
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.903131, total= 1.2min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.902941, total= 1.3min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.913386, total= 1.3min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.913519, total= 1.3min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.913947, total= 1.4min
[CV] C=100.0 .........................................................
[CV] ............................ C=1.0, score=0.905512, total= 1.4min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] ......................... C=1000.0, score=0.913894, total= 2.1min
[CV] C=100000.0 ......................................................
[CV] .......................... C=100.0, score=0.917331, total= 2.1min
[CV] C=100000.0 ......................................................
[CV] ......................... C=1000.0, score=0.903543, total= 2.0min
[CV] C=100000.0 ......................................................
[CV] ......................... C=1000.0, score=0.910803, total= 1.9min
[CV] C=100000.0 ......................................................
[CV] ......................... C=1000.0, score=0.907023, total= 2.0min
[CV] C=100000.0 ......................................................
[CV] ......................... C=1000.0, score=0.910433, total= 2.0min
[CV] C=1000000.0 .....................................................
[CV] ......................... C=1000.0, score=0.906746, total= 2.0min
[CV] C=1000000.0 .....................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:  5.3min


[CV] C=1000000.0 .....................................................
[CV] ........................ C=10000.0, score=0.918307, total= 2.0min
[CV] C=10000000.0 ....................................................
[CV] ....................... C=100000.0, score=0.903320, total= 1.9min
[CV] C=10000000.0 ....................................................
[CV] ........................ C=10000.0, score=0.904573, total= 2.0min
[CV] C=10000000.0 ....................................................
[CV] ........................ C=10000.0, score=0.913690, total= 2.0min
[CV] C=10000000.0 ....................................................
[CV] ....................... C=100000.0, score=0.912916, total= 2.0min
[CV] C=10000000.0 ....................................................
[CV] ........................ C=10000.0, score=0.919323, total= 2.1min
[CV] C=10000000.0 ....................................................
[CV] ....................... C=100000.0, score=0.894118, total= 2.0min
[CV] C

[Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:  7.0min


[CV] ...................... C=1000000.0, score=0.911937, total= 1.9min
[CV] C=100000000.0 ...................................................
[CV] ....................... C=100000.0, score=0.888446, total= 2.1min
[CV] C=100000000.0 ...................................................
[CV] ...................... C=1000000.0, score=0.889216, total= 2.0min
[CV] C=100000000.0 ...................................................
[CV] ...................... C=1000000.0, score=0.901088, total= 1.9min
[CV] C=100000000.0 ...................................................
[CV] ...................... C=1000000.0, score=0.768701, total= 2.0min
[CV] C=100000000.0 ...................................................
[CV] ...................... C=1000000.0, score=0.900591, total= 2.0min
[CV] C=100000000.0 ...................................................
[CV] ...................... C=1000000.0, score=0.913690, total= 2.0min
[CV] C=1000000000.0 ..................................................
[CV] .

In [15]:
top_model.cv_results_

{'mean_fit_time': array([ 60.79163394,  76.17706535, 102.22109761, 119.65016191,
        119.17047508, 120.25205352, 119.1917269 , 120.33910685,
        119.67981994, 119.34694273, 120.19270875, 117.22395129,
         95.918083  ]),
 'mean_score_time': array([0.02493327, 0.03045902, 0.02036474, 0.02138574, 0.01611872,
        0.02474458, 0.01777637, 0.02040918, 0.0258471 , 0.02822518,
        0.02312243, 0.01492891, 0.01319456]),
 'mean_test_score': array([0.9168311 , 0.91426598, 0.90814917, 0.90370955, 0.90588003,
        0.90706393, 0.9085438 , 0.88900947, 0.88723362, 0.88644436,
        0.90173639, 0.90341358, 0.90765588]),
 'mean_train_score': array([0.96144673, 0.9750722 , 0.97377873, 0.96799119, 0.96929378,
        0.97445913, 0.97437041, 0.95339771, 0.95114805, 0.95018095,
        0.968812  , 0.96994038, 0.97472149]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [16]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.91683, std: 0.00592, params: {'C': 0.01},
 mean: 0.91427, std: 0.00533, params: {'C': 0.1},
 mean: 0.90815, std: 0.00466, params: {'C': 1.0},
 mean: 0.90371, std: 0.00863, params: {'C': 10.0},
 mean: 0.90588, std: 0.00861, params: {'C': 100.0},
 mean: 0.90706, std: 0.00439, params: {'C': 1000.0},
 mean: 0.90854, std: 0.00783, params: {'C': 10000.0},
 mean: 0.88901, std: 0.04083, params: {'C': 100000.0},
 mean: 0.88723, std: 0.04046, params: {'C': 1000000.0},
 mean: 0.88644, std: 0.04038, params: {'C': 10000000.0},
 mean: 0.90174, std: 0.01010, params: {'C': 100000000.0},
 mean: 0.90341, std: 0.01045, params: {'C': 1000000000.0},
 mean: 0.90766, std: 0.00589, params: {'C': 10000000000.0}]

In [17]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.01} - score: 0.9168
